## FiTech: Data Driven Credit Card Design

* Team-lead GitHub userid: rsm-
* Group name: Group 40
* Team member names:
    - Kieran Huffman
    - Aarushi Gaikwad
    - Charles Qian
    - Sakshi Gandhi

## Setup

Please complete this python notebook with your group by answering the questions in `fitech-msba.pdf`.

Create a Notebook with all your results and comments and push the Notebook to GitHub when your team is done. Make sure to connect the GitHub repo to GradeScope before the due date. All results MUST be reproducible (i.e., the TA and I must be able to recreate your output from the Jupyter Notebook without changes or errors). This means that you should NOT use any python-packages that are not part of the RSM-MSBA docker container.

> Note: Please do not install any packages as part of your Jupyter Notebook submission

This is a group assignment and you will be using Git and GitHub. If two people edit the same file at the same time you could get what is called a "merge conflict". This is not something serious but you should realize that Git will not decide for you who's changes to accept. The team-lead will have to determine the edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and then push them to GitHub. Make "pull first" a habit!

If multiple people are going to work on the assignment at the same time I recommend you work in different notebooks. You can then `%run ...`  these "sub" notebooks from the main assignment file. You can seen an example of this in action below for the `model1.ipynb` notebook

Some group work-flow tips:

* Pull, edit, save, stage, commit, and push
* Schedule who does what and when
* Try to avoid working simultaneously on the same file
* If you are going to work simultaneously, do it in different notebooks, e.g.,
    - model1.ipynb, question1.ipynb, etc.
* Use the `%run ... ` command to bring different pieces of code together into the main jupyter notebook
* Put python functions in modules that you can import from your notebooks. See the example below for the `example` function defined in `utils/functions.py`

A graphical depiction of the group work-flow is shown below:

![](images/github-group-workflow.png)

In [1]:
import pandas as pd
import pyrsm as rsm

In [2]:
# Load the data
ex1 = pd.read_excel("data/exhibits.xlsx", sheet_name="exhibit1") # Exhibit 1: Results of Recent Solicitations
ex2 = pd.read_excel("data/exhibits.xlsx", sheet_name="exhibit2") # Exhibit 2: CLV Estimates for all 12 product offerings

In [3]:
ex1.head()

,date,apr,fixed_var,annual_fee,visamc,nr_mailed,non_resp,resp,bk_score
0,April,16.8,Fixed,20,MC,167000,165467,1533,200
1,April,16.8,Fixed,0,MC,81000,78104,2896,200
2,April,19.8,Fixed,20,MC,143000,142410,590,200
3,April,19.8,Fixed,0,MC,100000,97948,2052,200
4,September,14.9,Fixed,20,Visa,177000,172671,4329,250


In [4]:
ex2.head()

,offer,apr,fixed_var,annual_fee,clv150,clv200,clv250
0,1,14.9,Fixed,20,83,63,33
1,2,14.9,Variable,20,93,73,43
2,3,14.9,Fixed,0,52,32,2
3,4,14.9,Variable,0,62,42,12
4,5,16.8,Fixed,20,103,83,53


In [5]:
# No need to calculate "no sale" column as it's already included in dataset (ex1['non_resp'])

# Stack Exhibit 1 dataset
ex1_melt = pd.melt(
    ex1,
    id_vars=['date', 'apr', 'fixed_var', 'annual_fee', 'visamc', 'nr_mailed', 'bk_score'],
    value_vars=['non_resp', 'resp'],
    var_name='resp_type', # Categorical variable for response type: Did they respond? Or did they not respond?
    value_name='count'
)

ex1_melt.dtypes

# Fix categorical datatypes where necessary
ex1_melt['date'] = ex1_melt['date'].astype('category')
ex1_melt['fixed_var'] = ex1_melt['fixed_var'].astype('category')
ex1_melt['visamc'] = ex1_melt['visamc'].astype('category')

In [6]:
# Uncomment the last line and use the dataset it creates if not using weights="count" in logistic regression model

# Create 1.52 million row dataset for use in model building
# df = rsm.table2data(ex1_melt, 'count')

In [ ]:
# Perform logistic regression
lr = rsm.model.logistic(
    data={"Exhibit 1 (stacked)": ex1_melt},
    rvar="resp_type",
    lev="resp",
    evar=["apr", "fixed_var", "annual_fee", "visamc", "nr_mailed", "bk_score"],
    weights="count"
)

lr.summary()

Logistic regression (GLM)
Data                 : Exhibit 1 (stacked)
Response variable    : resp_type
Level                : resp
Explanatory variables: apr, fixed_var, annual_fee, visamc, nr_mailed, bk_score
Weights used         : count
Null hyp.: There is no effect of x on resp_type
Alt. hyp.: There is an effect of x on resp_type

                        OR     OR%  coefficient  std.error  z.value p.value     
Intercept            1.728   72.8%         0.55      0.081    6.752  < .001  ***
fixed_var[Variable]  0.813  -18.7%        -0.21      0.021   -9.977  < .001  ***
visamc[Visa ]        1.062    6.2%         0.06      0.018    3.425  < .001  ***
apr                  0.769  -23.1%        -0.26      0.005  -54.503  < .001  ***
annual_fee           0.943   -5.7%        -0.06      0.001  -54.148  < .001  ***
nr_mailed            1.000   -0.0%        -0.00      0.000   -2.869   0.004   **
bk_score             1.003    0.3%         0.00      0.000   12.459  < .001  ***

Signif. codes:  

## Preparation Questions

**1. Why does Customer Lifetime Value vary with BK score? Why does Customer Lifetime Value vary by 
product? (See Exhibit 2 to help answer these questions) (3 points)**

Customer lifetime value (CLV) varies with BK score because, the higher a customer's BK score is, the more likely they are to file for bankruptcy. In the event that a customer files for bankruptcy, financial losses are incurred by FiTech, so it makes sense that customers with a higher BK score would have a lower CLV to reflect the increased probability of bankruptcy and financial losses to FiTech when working with these customers.

CLV also varies by product because different products have different costs and revenue streams associated with them. For example, some cards have variable interest rates instead of fixed interest rates, which could make revenues less predictable for those products. Some products also have annual fees whereas others don't, which is a reliable increase in revenue for those products. As such, certain products have revenue streams which are either easier to predict or outright greater than others, which would lead to different CLVs for different products.

**2. Are predictive models estimated on historical data useful in this case? If so, why? If not, why not? (4 points)**

Yes, predictive models estimated on historical data would be useful in this case. Such data would be greatly helpful in seeing how customers have behaved in the past and which credit cards they were most likely to purchase and stick with. This data could then be used to predict how they will behave and which products they would enjoy in the future. However, historical data for individual customers is not available for this task, so instead we have to work with the data we have for previous marketing campaigns and possible product offerings.

**3. Is there a “best product” that will likely be preferred by all customers? If so, what is it? (3 points)**

Yes, there is a "best product" that will likely be preferred by all customers, and that product is Offer 3. Offer 3 is the "best product" because it has an APR of 14.9% (the lowest of the APRs offered), no annual fee, and a fixed APR. This makes it the most affordable credit card for customers, especially with regard to the APR being in the lowest possible offering and the lack of an annual fee. Some customers may prefer a variable APR, but the fixed APR is a more reliable and predictable, making it the more attractive option for most customers. However, because this offering is the best deal for customers, it also has the lowest CLV values, at 52, 32, and 2 for customers with BK scores of 150, 200, and 250 respectively. As such, while offering this credit card would likely lead to the highest response rates from customers, it would also lead to the lowest CLV values for FiTech.

**4. Describe and justify your testing strategy (10 points)**

Our approach will involve running a partial factorial design to test various combinations of APR, annual fee, and fixed versus variable rates. We will first use the seed value 1234 which ensures reproducibility, which is useful when comparing different experimental runs. If needed, we can test different seeds to see how the design varies. Additionally, sample size considerations should be addressed, ensuring statistical significance in the results. The guideline suggests using 4,000 samples per cell, but deviations can be justified based on response variability.

Next, we should outline the key metrics for evaluation. Response rate will measure the percentage of customers who accept the credit card offer, while customer lifetime value (CLV) will determine the profitability of each acquired customer. Default rate analysis will be essential to identify whether certain product features attract riskier customers, and net profit will be calculated after accounting for fixed and variable costs associated with the campaign.

The initial test allocation plan should be structured to maximize insight. A balanced testing approach could be used by distributing offers equally among the BK score groups, or a risk-weighted testing strategy could prioritize lower-risk segments, as they are more likely to generate long-term profitability. This allocation will help determine which product features lead to the highest CLV while minimizing the risk of defaults.

To refine our submission from Round 1 to Round 2, we should closely analyze the response rates and profitability metrics from the initial test. If certain products performed well within specific BK score segments, we should scale up those offerings in Round 2 to maximize overall profitability. Conversely, if some products had low response rates or resulted in negative net profit due to high default risk, we should consider eliminating or reducing their distribution. Additionally, if response rates for high-APR or high-fee products were lower than expected, we may need to adjust our allocation by offering these to lower-risk customers (BK 150) who have a higher likelihood of long-term profitability. Another key adjustment is evaluating the performance of fixed versus variable-rate products; if variable-rate cards were significantly less attractive to customers despite their ability to mitigate interest rate risk, we may need to prioritize fixed-rate offerings in Round 2. Given that Round 2 is the final rollout, our focus will be on maximizing profit rather than just gathering insights in Round 1. This means strategically distributing solicitations to ensure the highest CLV per acquired customer while keeping costs under control. If a competitor's influence was observed in Round 1, such as a shift in customer preference toward no-fee or low-APR cards, we should take that into account when finalizing our Round 2 allocations. By making data-driven adjustments and optimizing product allocation, we can enhance profitability and ensure that the solicitation strategy is aligned with market dynamics and customer behavior.

**5. Generative AI (5 points)**

We used ChatGPT to support our understanding of the FiTech case by refining our testing strategy, evaluating design factors, and optimizing product allocation. For example we asked ChatGPT how should we best adjust our Round 2 submission based on Round 1 results. This helped clarify the decision-making process for maximizing profitability. Additionally, we asked for guidance on factorial design in Radiant, and ChatGPT suggested including BK score as a factor, which improved the robustness of our experimental setup. While the AI responses were helpful in structuring our approach, one limitation was the lack of direct integration with Radiant-for-R, meaning we had to manually validate the design choices within the tool. Another challenge was that AI-generated suggestions sometimes required refinement based on case-specific constraints, such as limited solicitation rounds. A key insight from using Generative AI was its ability to quickly summarize complex concepts and provide alternative strategies, allowing us to iterate on our approach efficiently. However, we also learned the importance of critically evaluating AI-generated recommendations and cross-referencing them with the case study data to ensure accurate decision-making.

### SIMULATION EXERCISE

In [15]:
import pandas as pd

file_path = "data/part_factorial.csv"
df = pd.read_csv(file_path)

# Display the first few rows to understand its structure
df.head()

,trial,APR,Annual_fee,FixedVariable,Product_Offer,bk_score
0,7,14.0,0,Fixed,3,150
1,21,14.0,0,Fixed,7,250
2,23,14.0,0,Fixed,8,200
3,27,14.0,0,Fixed,9,250
4,28,14.0,0,Fixed,10,150


In [16]:
# Summarize the count of each product offer per BK score group
product_distribution = df.groupby(["Product_Offer", "bk_score"]).size().unstack(fill_value=0)

# Calculate the total occurrences of each product in the experimental design
product_distribution["Total"] = product_distribution.sum(axis=1)

# Normalize to determine proportion of emails to allocate
product_distribution["150_pct"] = product_distribution[150] / product_distribution["Total"]
product_distribution["200_pct"] = product_distribution[200] / product_distribution["Total"]
product_distribution["250_pct"] = product_distribution[250] / product_distribution["Total"]

# Calculate the recommended number of emails to send per product and BK group
total_emails = 750000  # Total emails available
emails_per_product = total_emails / len(product_distribution)  # Even distribution

product_distribution["Emails_150"] = (emails_per_product * product_distribution["150_pct"]).astype(int)
product_distribution["Emails_200"] = (emails_per_product * product_distribution["200_pct"]).astype(int)
product_distribution["Emails_250"] = (emails_per_product * product_distribution["250_pct"]).astype(int)

print(product_distribution) 


bk_score       150  200  250  Total  150_pct  200_pct  250_pct  Emails_150  \
Product_Offer                                                                
1                2    1    2      5      0.4      0.2      0.4       25000   
2                2    1    2      5      0.4      0.2      0.4       25000   
3                2    2    1      5      0.4      0.4      0.2       25000   
4                2    1    2      5      0.4      0.2      0.4       25000   
5                1    2    2      5      0.2      0.4      0.4       12500   
6                1    2    2      5      0.2      0.4      0.4       12500   
7                1    2    2      5      0.2      0.4      0.4       12500   
8                2    1    2      5      0.4      0.2      0.4       25000   
9                2    2    1      5      0.4      0.4      0.2       25000   
10               2    2    1      5      0.4      0.4      0.2       25000   
11               2    2    1      5      0.4      0.4      0.2  

For Round 1, we designed our email allocation using a proportional allocation strategy based on a partial factorial design we made in Radiant. The goal was to ensure that each product offer was tested across different BK score segments (150, 200, 250) to gather insights on customer response behavior. Initially, we distributed emails evenly across all offers, allocating 750,000 emails proportionally to ensure each product received a fair test. This method allowed us to observe how different combinations of APR, annual fees, and fixed vs. variable rates performed across varying credit risk levels. Since we did not have prior data on response rates or profitability, the Round 1 allocation served as an exploratory test to identify high- and low-performing offers before making data-driven adjustments for the final rollout.

In [17]:
df_round1 = pd.read_csv("data/Round1_Results.csv")

# Rename columns for easier manipulation
df_round1.rename(columns={
    "Product Number": "Product",
    "150 Sent": "Sent_150", "150 Responses": "Responses_150",
    "200 Sent": "Sent_200", "200 Responses": "Responses_200",
    "250 Sent": "Sent_250", "250 Responses": "Responses_250"
}, inplace=True)

# Calculate response rates
df_round1["ResponseRate_150"] = df_round1["Responses_150"] / df_round1["Sent_150"]
df_round1["ResponseRate_200"] = df_round1["Responses_200"] / df_round1["Sent_200"]
df_round1["ResponseRate_250"] = df_round1["Responses_250"] / df_round1["Sent_250"]

# Define hypothetical CLV values for each BK segment (higher CLV for lower BK)
clv_150 = 120  # Higher CLV due to lower risk
clv_200 = 90
clv_250 = 60  # Lower CLV due to higher default risk

# Calculate weighted CLV-adjusted response scores
df_round1["Weighted_150"] = df_round1["ResponseRate_150"] * clv_150
df_round1["Weighted_200"] = df_round1["ResponseRate_200"] * clv_200
df_round1["Weighted_250"] = df_round1["ResponseRate_250"] * clv_250

# Identify low-performing products based on response rates
low_threshold = 0.005  # 0.5% response rate cutoff
eliminate_products = df_round1[
    (df_round1["ResponseRate_150"] < low_threshold) &
    (df_round1["ResponseRate_200"] < low_threshold) &
    (df_round1["ResponseRate_250"] < low_threshold)
]["Product"].tolist()

# Remove low-performing products from the allocation
df_filtered = df_round1[~df_round1["Product"].isin(eliminate_products)]

# Normalize weights again after removal
total_weight = df_filtered["Weighted_150"].sum() + df_filtered["Weighted_200"].sum() + df_filtered["Weighted_250"].sum()
df_filtered["Alloc_150"] = (df_filtered["Weighted_150"] / total_weight) * 750000
df_filtered["Alloc_200"] = (df_filtered["Weighted_200"] / total_weight) * 750000
df_filtered["Alloc_250"] = (df_filtered["Weighted_250"] / total_weight) * 750000

# Convert to integers for final allocation
df_filtered["Alloc_150"] = df_filtered["Alloc_150"].astype(int)
df_filtered["Alloc_200"] = df_filtered["Alloc_200"].astype(int)
df_filtered["Alloc_250"] = df_filtered["Alloc_250"].astype(int)

# Prepare final allocation table
final_allocation_filtered = df_filtered[["Product", "Alloc_150", "Alloc_200", "Alloc_250"]]
final_allocation_filtered


/tmp/ipykernel_65845/4168718170.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Alloc_150"] = (df_filtered["Weighted_150"] / total_weight) * 750000
/tmp/ipykernel_65845/4168718170.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Alloc_200"] = (df_filtered["Weighted_200"] / total_weight) * 750000
/tmp/ipykernel_65845/4168718170.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,Product,Alloc_150,Alloc_200,Alloc_250
0,1,23065,25283,20150
1,2,16221,15968,12926
2,3,60197,117955,25061
3,4,55825,53322,45655
4,5,7477,7318,5671
5,6,4055,5132,4213
6,7,23445,24094,19168
7,8,22558,25187,16918
10,11,19960,31936,13180
11,12,17108,17631,13306


For Round 2, we refined our allocation strategy based on the results from Round 1, incorporating both response rates and Customer Lifetime Value (CLV) to maximize profitability. We first calculated response rates for each product across BK score segments and identified high-performing, moderate-performing, and low-performing products. Products with very low response rates across all BK scores were removed to optimize the allocation for more effective offers. We then adjusted our email distribution using a CLV-weighted response model, which prioritized BK 150 customers due to their higher CLV and lower default risk, while still considering BK 200 and BK 250 where response rates were strong. This final allocation ensures that we are maximizing overall profitability, rather than just increasing the number of responses, by strategically distributing emails to segments and products with the highest expected return.